In [1]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os
import numpy
import itertools
MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")

saved_csv folder already exists.


#### The following are special functions used by the classifier.  Don't change them.

In [2]:
def find_entry_int_distance(coordinates, piece: intervals.main_objs.ImportedPiece):
    tone_list = []
    all_tones = piece.getNoteRest()
    
    for item in coordinates:
        filtered_tones = all_tones.loc[item] 
        tone_list.append(filtered_tones)
        
    noteObjects = [note.Note(tone) for tone in tone_list]
    _ints = [interval.Interval(noteObjects[i], noteObjects[i + 1]) for i in range(len(noteObjects) - 1)]
    entry_ints = []
    
    for _int in _ints:
        entry_ints.append(_int.directedName)
    
    return entry_ints

def split_by_threshold(seq, max_diff=70):  
    it = iter(seq)
    last = next(it)
    part = [last]

    for curr in it:
        if curr - last > max_diff:
            yield part
            part = []

        part.append(curr)
        last = curr
#         print(part)
        
    yield part
    

def classify_by_offset(offset_diffs):
    """
    Put logic for classifying an offset list here
    """
    #
#     offset_difference_list = points['Time_Entry_Intervals']

    alt_list = offset_diffs[::2]

    if len(set(offset_diffs)) == 1 and len(offset_diffs) > 1:
        return 'PEN'
    # elif (len(offset_difference_list) %2 != 0) and (len(set(alt_list)) == 1):
    elif (len(offset_diffs) % 2 != 0) and (len(set(alt_list)) == 1) and (len(offset_diffs) >= 3):
        return 'ID'
    elif len(offset_diffs) >= 1:
        return 'FUGA'


#### Load the Piece Here

* Note that you can load from CRIM, or put a file in the **Music_Files** folder in the Notebook.

In [3]:

piece = importScore('https://crimproject.org/mei/CRIM_Model_0008.mei')
# piece = importScore('Music_Files/CRIM_Mass_0007_4.mei')


Successfully imported https://crimproject.org/mei/CRIM_Model_0008.mei


#### Run the Classifier Here

In [4]:
# THIS WORKS 3/1/22

points = pd.DataFrame()
# new_offset_list = []
nr = piece.getNoteRest()
det = piece.detailIndex(nr, offset=True)

# durations and ngrams of durations
dur = piece.getDuration(df=nr)
dur_ng = piece.getNgrams(df=dur, n=4)

# ngrams of melodic entries
# for chromatic, use:
# piece.getMelodicEntries(interval_settings=('c', True, True), n=5)
mel = piece.getMelodicEntries(n=4)
mels_stacked = mel.stack().to_frame()
mels_stacked.rename(columns =  {0:"pattern"}, inplace = True)

# edit distance, based on side-by-side comparison of melodic ngrams
# gets flexed and other similar soggetti
dist = piece.getDistance(mel)
dist_stack = dist.stack().to_frame()


# filter distances to threshold.  <2 is good
filtered_dist_stack = dist_stack[dist_stack[0] < 2]
filtered_dist = filtered_dist_stack.reset_index()
filtered_dist.rename(columns =  {'level_0':"source", 'level_1':'match'}, inplace = True)

# Group the filtered distanced patterns
full_list_of_matches = filtered_dist.groupby('source')['match'].apply(list).reset_index()

for matches in full_list_of_matches["match"]:
    related_entry_list = mels_stacked[mels_stacked['pattern'].isin(matches)]
    entry_array = related_entry_list.reset_index(level=1).rename(columns = {'level_1': "voice", 0: "pattern"})
#     voice_list = entry_array['voice'].to_list()
    offset_list = entry_array.index.to_list()
    split_list = list(split_by_threshold(offset_list))
    for item in split_list:
#         print(item)
        array = entry_array[entry_array.index.get_level_values(0).isin(item)]
        short_offset_list = array.index.to_list()
        time_ints = numpy.diff(array.index).tolist()
        voice_list = array['voice'].to_list()
# time_ints
        tone_coordinates =  list(zip(short_offset_list, voice_list))
# tone_coordinates.ffill(inplace=True)
        mel_ints = find_entry_int_distance(tone_coordinates, piece)
        first_offset = short_offset_list[0]
# #     first_note_list = nr[offset_list]
        meas_beat = det[det.index.get_level_values('Offset').isin(short_offset_list)]
        mb2 = meas_beat.reset_index()
        mb2['mb'] = mb2["Measure"].astype(str) + "/" + mb2["Beat"].astype(str)
        meas_beat_list = mb2['mb'].to_list()


        temp = {'First_Offset': first_offset, 
                    'Offsets': short_offset_list, 
                    'Measures_Beats': meas_beat_list,
                    "Soggetti": matches,
                    'Voices': voice_list, 
                    'Time_Entry_Intervals': time_ints, 
                    'Melodic_Entry_Intervals': mel_ints}
        
        points = points.append(temp, ignore_index=True).sort_values("First_Offset")
        points = points[points['Offsets'].apply(len) > 1]
        points.drop_duplicates(subset=["First_Offset"], keep='first', inplace = True)
        points['Presentation_Type'] = points['Time_Entry_Intervals'].apply(classify_by_offset)
points['Number_of_Entries'] = points['Offsets'].apply(len)

points = points[['First_Offset',
        'Measures_Beats',
        'Presentation_Type', 
        "Number_of_Entries", 
        'Melodic_Entry_Intervals',
        'Time_Entry_Intervals',
        'Voices','Soggetti']]
points


# points_no_singles = points_clean['Soggetti'].apply(lambda x: ('\n'.join(x)))



,First_Offset,Measures_Beats,Presentation_Type,Number_of_Entries,Melodic_Entry_Intervals,Time_Entry_Intervals,Voices,Soggetti
0,0.0,"[1/1.0, 3/1.0, 5/1.0, 7/1.0]",PEN,4,"[P-8, P1, P-8]","[16.0, 16.0, 16.0]","[[Superius], Altus, Tenor, Bassus]","[4, 1, 2, 2]"
1,56.0,"[8/1.0, 10/1.0, 12/1.0, 14/1.0]",PEN,4,"[P-8, P1, P-8]","[16.0, 16.0, 16.0]","[[Superius], Altus, Tenor, Bassus]","[-2, -2, -2, 2]"
2,124.0,"[16/3.0, 18/3.0, 20/3.0, 22/3.0]",PEN,4,"[P-8, P1, P-8]","[16.0, 16.0, 16.0]","[[Superius], Altus, Tenor, Bassus]","[1, 1, 2, 2, 1, 1, 1, 2]"
3,176.0,"[23/1.0, 28/1.0]",FUGA,2,[P-8],[40.0],"[Altus, Bassus]","[-3, -2, 2, 2]"
4,244.0,"[31/3.0, 35/3.0]",FUGA,2,[P-8],[32.0],"[[Superius], Tenor]","[1, 1, 2, 2, 1, 2, 2, 1, 1, 1, 2, 1]"
5,316.0,[40/3.0],FUGA,2,[M-6],[0.0],"[[Superius], Tenor]","[1, 1, 2, -2, 1, 1, 2, -3]"
6,428.0,"[54/3.0, 55/1.0, 59/3.0, 60/1.0]",ID,4,"[P-5, P-4, P-5]","[4.0, 36.0, 4.0]","[[Superius], Altus, Tenor, Bassus]","[4, -2, 2, 2]"
7,508.0,"[64/3.0, 66/3.0, 67/3.0, 69/3.0]",ID,4,"[P-5, P-4, P-5]","[16.0, 8.0, 16.0]","[[Superius], Altus, Tenor, Bassus]","[2, -3, 2, -3]"
8,616.0,"[78/1.0, 81/1.0, 84/1.0]",PEN,3,"[P-8, M7]","[24.0, 24.0]","[[Superius], Tenor, [Superius]]","[1, 1, 2, -2, 1, 1, 2, -3]"
9,748.0,"[94/1.0, 98/1.0]",FUGA,2,[P1],[48.0],"[Altus, Altus]","[1, 1, 2, -2, 1, 1, 2, -3]"


#### Save to your folder of CSV's here in the Jupyter Hub
#### You can save as CSV, or as Excel.
#### You will then need to download this to your computer to view it properly

* Note that in the following part of the code below, you will need to give your file a name:

`saved_csv/**file_name**.xlsx`

In [5]:
writer = pd.ExcelWriter('saved_csv/file_name.xlsx', engine='xlsxwriter')
points.to_excel(writer, sheet_name='Sheet1')
writer.save()

In [6]:
points.to_csv('saved_csv/your_file_title.csv')